In [2]:
import pandas as pd
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/challenge.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.19,168.86,52.00,broken clouds,0.00,0.0,92,77,4.00,NZ,2020-02-23 19:39:18
1,1,Borba,-4.39,-59.59,83.37,light rain,1.56,0.0,87,74,3.22,BR,2020-02-23 19:40:01
2,2,Jamestown,42.10,-79.24,48.00,clear sky,0.00,0.0,24,1,16.11,US,2020-02-23 19:32:43
3,3,Smithers,54.78,-127.17,26.60,broken clouds,0.00,0.0,68,75,2.24,CA,2020-02-23 19:39:56
4,4,Rikitea,-23.12,-134.97,79.29,light rain,0.75,0.0,73,6,10.96,PF,2020-02-23 19:32:47


In [3]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
    
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = str(input("Do you want it to be raining on your trip? (yes/no) "))
snow = str(input("Do you want it to be snowing on your trip? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining on your trip? (yes/no) yes
Do you want it to be snowing on your trip? (yes/no) no


In [4]:
wantsRain = False
if(rain == 'yes'):
    
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Min Temp"] >= min_temp) &\
                                       (city_data_df["Rain inches (last 3 hrs)"] == 0)
                                       (city_data_df["Snow inches (last 3 hrs)"] ==0)]


preferred_cities_df.head()
                                                 


,City_ID,City,Lat,Lng,Max Temp,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs),Humidity,Cloudiness,Wind Speed,Country,Date
5,5,Hilo,19.73,-155.09,75.99,broken clouds,0.0,0.0,60,75,4.70,US,2020-02-23 19:40:02
7,7,Souillac,-20.52,57.52,77.00,broken clouds,0.0,0.0,88,75,4.70,MU,2020-02-23 19:35:57
8,8,Vaini,-21.20,-175.20,78.80,scattered clouds,0.0,0.0,94,40,8.05,TO,2020-02-23 19:40:02
12,12,Georgetown,5.41,100.34,80.60,few clouds,0.0,0.0,74,20,3.36,MY,2020-02-23 19:32:45
19,19,Mogadishu,2.04,45.34,80.42,clear sky,0.0,0.0,82,0,27.58,SO,2020-02-23 19:35:40


In [5]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [6]:
import gmaps
import requests
# Import the API key.
from config import g_key
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [7]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Add a marks of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))